# Kafkaのトピック(pyspark-topic10)を作成してみましょう

```
/home/pyspark/kafka/bin/kafka-topics.sh \
    --create --topic pyspark-topic10 \
    --replication-factor 1 \
    --partitions 1 \
    --bootstrap-server kafka:9092 
```

# 後方互換
古いスキーマバージョンでシリアライズされたデータを新しいスキーマバージョンでコンシュームすることが可能にすることです。

In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-avro_2.12:3.2.0") \
    .enableHiveSupport() \
    .getOrCreate()

# パッケージを複数渡したい時は「,」で繋いで渡します。
# Sparkのバージョンにしっかりと合わせます(今回はSparkのバージョンが3.2を使っています。)。

In [ ]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("subscribe", "pyspark-topic10") \
  .load()

In [ ]:
from confluent_kafka import Producer
import avro.schema
import avro.io
import io
import random

# スキーマファイルを読み込んでおきます
avro_json_schema = open("/home/pyspark/pyspark_streaming/schema/schema_ver1.avsc", "r").read()
avro_json_schema_ver2 = open("/home/pyspark/pyspark_streaming/schema/schema_ver2.avsc", "r").read()

In [ ]:
# 送信するpyspark 
# ただし古いバージョンで送ります

conf = {'bootstrap.servers': 'kafka:9092'}
producer = Producer(**conf)

# Kafka topic
topic = "pyspark-topic10"

# Path to user.avsc avro schema
schema_path = "/home/pyspark/pyspark_streaming/schema/schema_ver1.avsc"
schema = avro.schema.parse(open(schema_path).read())

for i in range(1):
    writer = avro.io.DatumWriter(schema)
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    # データの送信
    writer.write({"id": "yuki_schemaver1",
                    "type": "login2",
                    "sendtime": random.randint(0, 10)}, encoder)
    raw_bytes = bytes_writer.getvalue()
    producer.produce(topic, raw_bytes, "1")
producer.flush()

In [ ]:
# 送信するpyspark 
# スキーマバージョン2で送信します

conf = {'bootstrap.servers': 'kafka:9092'}
producer_ver2 = Producer(**conf)

# Kafka topic
topic = "pyspark-topic10"

# Path to user.avsc avro schema
schema_path_ver2 = "/home/pyspark/pyspark_streaming/schema/schema_ver2.avsc"
schema_ver2 = avro.schema.parse(open(schema_path_ver2).read())

for i in range(1):
    writer = avro.io.DatumWriter(schema_ver2)
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    # データの送信
    writer.write({
                    "id": "ver2_yuki_schema",
                    "type": "login2",
                    "sendtime": random.randint(0, 10),
                    "temp": "1000"
                }, encoder)
    raw_bytes = bytes_writer.getvalue()
    producer_ver2.produce(topic, raw_bytes, "2")
producer_ver2.flush()

In [ ]:
# spark streaming(read avro) -> memory
# スキーマバージョン2でデータ読み取ります(後方互換です。)
from pyspark.sql.avro.functions import from_avro, to_avro

memory_stream_check = df \
  .select(df.key,from_avro("value", avro_json_schema_ver2, {"mode" : "PERMISSIVE"}).alias("json_col")) \
  .writeStream \
  .trigger(processingTime="5 seconds") \
  .format("memory") \
  .queryName("ensyu2") \
  .option("checkpointLocation", "/tmp/kafka/backword_check_ensyu2/") \
  .start()


In [ ]:
spark.sql("select * from ensyu2")